Ahmet Alperen Gungor, 28847
Elifnur Ozturk, 28857 
Ayse Sena Acar, 29506
Baris Bakirdoven, 29028

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from keras.datasets import cifar10
from sklearn.metrics import accuracy_score
import numpy as np
import pickle
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount("./drive")

Mounted at ./drive


In [ ]:
# Function to load each batch file
def load_cifar_batch(file):
    with open("./drive/My Drive/dataset/" + file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict[b'data'], dict[b'labels']

# Define batch files
batch_files = ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']

# Perform 5-fold cross-validation
accuracies = []

for i in range(5):
    print(f"Starting fold {i+1}")
    
    # Load validation data from the current batch file
    x_val, y_val = load_cifar_batch(batch_files[i])
    x_val = x_val.reshape(-1, 3, 32, 32).transpose(0,2,3,1) / 255.0
    y_val = np.array(y_val)
    
    # Load training data from the other batch files
    x_train = []
    y_train = []
    for j in range(5):
        if j != i:
            x_batch, y_batch = load_cifar_batch(batch_files[j])
            x_batch = x_batch.reshape(-1, 3, 32, 32).transpose(0,2,3,1) / 255.0
            x_train.append(x_batch)
            y_train.append(y_batch)
    x_train = np.concatenate(x_train)
    y_train = np.concatenate(y_train)


    X_train = x_train.reshape(x_train.shape[0], -1)
    X_val = x_val.reshape(x_val.shape[0], -1)

    for n_neighbor in range(1,10):
      if(n_neighbor%2 == 1):
        knn_model = KNeighborsClassifier(n_neighbors = n_neighbor)
        knn_model.fit(X_train, y_train)
        
        knn_preds = knn_model.predict(X_val)
        print("k = ", n_neighbor, ", Accuracy: ", accuracy_score(y_val, knn_preds))

Starting fold 1
k =  1 , Accuracy:  0.3358
k =  3 , Accuracy:  0.326
k =  5 , Accuracy:  0.335
k =  7 , Accuracy:  0.34
k =  9 , Accuracy:  0.3414
Starting fold 2
k =  1 , Accuracy:  0.3361
k =  3 , Accuracy:  0.3261
k =  5 , Accuracy:  0.3297
k =  7 , Accuracy:  0.333
k =  9 , Accuracy:  0.327
Starting fold 3
k =  1 , Accuracy:  0.3469
k =  3 , Accuracy:  0.328
k =  5 , Accuracy:  0.3336
k =  7 , Accuracy:  0.3356
k =  9 , Accuracy:  0.3345
Starting fold 4
k =  1 , Accuracy:  0.3346
k =  3 , Accuracy:  0.3252
k =  5 , Accuracy:  0.334
k =  7 , Accuracy:  0.331
k =  9 , Accuracy:  0.3368
Starting fold 5
k =  1 , Accuracy:  0.3383
k =  3 , Accuracy:  0.3178
k =  5 , Accuracy:  0.3283
k =  7 , Accuracy:  0.3237
k =  9 , Accuracy:  0.3223


In [ ]:
# Function to load each batch file
def load_cifar_batch(file):
    with open("./drive/My Drive/dataset/" + file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict[b'data'], dict[b'labels']

# Define batch files
batch_files = ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']

k1_avg = 0.33834 #k=1 is best
k3_avg = 0.32462
k5_avg = 0.33212
k7_avg = 0.33266
k9_avg = 0.3324



# Load the test batch
test_batch = load_cifar_batch('test_batch')

X_test = test_batch[0].reshape(-1, 3, 32, 32).transpose(0,2,3,1) / 255.0
y_test = np.array(test_batch[1])


# Load, reshape and combine all the training data batches
X_train_combined = np.concatenate([load_cifar_batch(batch)[0].reshape(-1, 3, 32, 32).transpose(0,2,3,1) / 255.0 for batch in batch_files])
y_train_combined = np.concatenate([np.array(load_cifar_batch(batch)[1]) for batch in batch_files])


# train the final KNN model using the best k value
knn_model_final = KNeighborsClassifier(n_neighbors=1)
knn_model_final.fit(X_train_combined.reshape(X_train_combined.shape[0], -1), y_train_combined)

# make predictions on the test data
knn_preds_final = knn_model_final.predict(X_test.reshape(X_test.shape[0], -1))

# compute the final accuracy
final_accuracy = accuracy_score(y_test, knn_preds_final)
print(f"Final accuracy on test data: {final_accuracy}")


Final accuracy on test data: 0.3539
